In [ ]:
# Current directory
import os
os.chdir('E:\Experiment\work')

### 导入数据


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split

X_new = pd.read_csv(r'G:/lhy\work/train_feature/feature\\ESMC300+AAC+PAAC.csv', header=None)
y_new = pd.read_csv(r'G:\\lhy\work\data/label_train.csv', header=None)

X_new1 = pd.read_csv(r"G:\\lhy\work\\test_feature\\ESMC300+AAC+PAAC.csv", header=None)
y_new1 = pd.read_csv(r'G:\\lhy\work\data/label_test.csv', header=None)


print(X_new.shape)
print(y_new.shape)
print(X_new1.shape)
print(y_new1.shape)

X_new = np.array(X_new)
y_new = np.array(y_new).ravel()

# X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# X_new1 = np.array(X_new1)
# y_new1 = np.array(y_new1).ravel()


(2872, 1003)
(2872, 1)
(342, 1003)
(342, 1)


In [ ]:
from sklearn.preprocessing import StandardScaler

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)


### 寻找最佳参数

In [ ]:
# dataset splitting
from sklearn.model_selection import train_test_split
X_train_whole, X_ind_test, y_train_whole, y_ind_test =  train_test_split( X_new, y_new, test_size=0.2, random_state=1111)
print(X_train_whole.shape)
print(X_ind_test.shape)

In [2]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# 基础参数
# base_params = {
#     'random_state': 42,
#     'n_jobs': -1,
#     'class_weight': {0: 1627, 1: 1245}
# }

# 参数搜索范围
param_grid = {
    'n_estimators': (100,400,10),
    'max_depth': (10,35,1),
    # 'num_leaves': [31, 63, 127],
    # 'learning_rate': [0.01, 0.03, 0.05, 0.1],
    # 'min_child_samples': [10, 20, 30]
}

# 模型和交叉验证配置
model = LGBMClassifier()
cv = StratifiedKFold(n_splits=5)

# 网格搜索
grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_search.fit(X_new, y_new)

# 输出最佳参数和得分
print("最佳参数:", grid_search.best_params_)
print("最佳AUC得分:", grid_search.best_score_)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[LightGBM] [Info] Number of positive: 1245, number of negative: 1627
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 251234
[LightGBM] [Info] Number of data points in the train set: 2872, number of used features: 1003
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.433496 -> initscore=-0.267602
[LightGBM] [Info] Start training from score -0.267602
最佳参数: {'max_depth': 35, 'n_estimators': 400}
最佳AUC得分: 0.7149671342282768


### 测试

In [5]:
import math
import joblib
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, log_loss

# 拆分数据
# X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# ExtraTreesClassifier 参数
et_params =  {
    'n_estimators': 200,      # 增加树的数量
    'max_depth': 21,          # 增加深度
    # 'min_samples_split': 7,   # 防止过拟合
    # 'min_samples_leaf': 1,   # 增加叶子节点的最小样本数
    'max_features': 'sqrt',   # 每棵树使用的特征比例
    # 'class_weight': 'balanced', # 处理类别不平衡
    # 'oob_score': True,  # 关键修复：启用OOB评估
    'n_jobs': -1,
    'class_weight': {0:1627, 1: 1245},  
    'random_state': 42
}

# 创建包含标准化的管道
model = make_pipeline(
    # StandardScaler(),  # 如果需要标准化，可以取消注释
    ExtraTreesClassifier(**et_params)
)

# 使用5折交叉验证训练和预测
cv = StratifiedKFold(n_splits=5)
y_pred = cross_val_predict(model, X_new, y_new, cv=cv, method='predict')
y_proba = cross_val_predict(model, X_new, y_new, cv=cv, method='predict_proba')[:, 1]

# 模型拟合（为了提取特征重要性）
model.fit(X_new, y_new)

# 安全除法函数
def safe_divide(a, b):
    return a / b if b != 0 else 0

# 评估指标
cm = confusion_matrix(y_new, y_pred)
TN, FP, FN, TP = cm.ravel()

AAC = (TP + TN) / (TP + TN + FP + FN)
Sn = safe_divide(TP, TP + FN)
Sp = safe_divide(TN, TN + FP)
MCC_num = (TP * TN - FP * FN)
MCC_den = math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
MCC = safe_divide(MCC_num, MCC_den)
AUC = roc_auc_score(y_new, y_proba)
logloss = log_loss(y_new, y_proba)

# 打印结果
print("\n========== 交叉验证评估报告（ExtraTreesClassifier） ==========")
print(f"准确率 (AAC): {AAC:.4f}")
print(f"灵敏度 (Sn): {Sn:.4f}")
print(f"特异度 (Sp): {Sp:.4f}")
print(f"马修斯系数 (MCC): {MCC:.4f}")
print(f"AUC分数: {AUC:.4f}")
print(f"对数损失: {logloss:.4f}")
print("混淆矩阵:")
print(cm)

# 特征重要性输出
et_model = model.named_steps['extratreesclassifier']
importances = et_model.feature_importances_

print("\nTop 10重要特征:")
sorted_idx = importances.argsort()[::-1]
feature_names = X_new.columns if hasattr(X_new, 'columns') else [f'Feature_{i}' for i in range(X_new.shape[1])]
for i in sorted_idx[:10]:
    print(f"{feature_names[i]}: {importances[i]:.4f}")


========== 交叉验证评估报告（ExtraTreesClassifier） ==========
准确率 (AAC): 0.6382
灵敏度 (Sn): 0.3952
特异度 (Sp): 0.8242
马修斯系数 (MCC): 0.2446
AUC分数: 0.6684
对数损失: 0.6382
混淆矩阵:
[[1341  286]
 [ 753  492]]

Top 10重要特征:
Feature_990: 0.0022
Feature_969: 0.0022
Feature_203: 0.0021
Feature_753: 0.0020
Feature_263: 0.0019
Feature_681: 0.0019
Feature_843: 0.0019
Feature_160: 0.0018
Feature_981: 0.0018
Feature_736: 0.0018


In [6]:

# 独立测试
import math
import numpy as np
import statistics
# import matplotlib.pyplot as plt
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,roc_auc_score
# from sklearn.model_selection import train_test_split

# result collection list
# BACC_collection = []
AAC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []
# AP = []
# mean_recall = np.linspace(0, 1, 100)
# all_precision = []
# base_fpr = np.linspace(0, 1, 100)
# mean_tpr = 0.0
# interp_tpr_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(5):
    # dataset splitting
    # X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)
    
    # Create and train Decision Tree classifier
    # clf = RandomForestClassifier(max_depth=600, max_features=1, n_estimators=1000, n_jobs=-1)  # need to tune the hyperparameters
    # clf.fit(X_train_whole, y_train_whole)
    # clf = joblib.load('mode\RF_ESM300/rf_ESM300_model_fold_5.joblib')
    # Predict on independent test set
    model.fit(X_new, y_new)
    y_pred_score = model.predict_proba(X_new1)
    y_pred = categorical_probas_to_classes(y_pred_score)
    y_true = y_new1
    TP, FP, FN, TN = confusion_matrix(y_true, y_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    AAC_collection.append((TP + TN) / (TP + TN + FP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP * TN - FP * FN) / math.pow(((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)), 0.5)
    MCC_collection.append(MCC)
    auc = roc_auc_score(y_true, y_pred_score[:, 1])
    AUC_collection.append(auc)
  
# Print results
print(round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print(round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print(round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print(round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print(round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))


0.632 ± 0.0
0.596 ± 0.0
0.71 ± 0.0
0.284 ± 0.0
0.684 ± 0.0


In [ ]:
import math
import numpy as np
import statistics
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score

# 初始化指标收集
AAC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []

# 保存预测结果
y_true_all = []
y_pred_all = []
y_pred_proba_all = []

# 概率转标签函数
def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

# ExtraTrees 分类器
clf = ExtraTreesClassifier(max_depth=13, n_estimators=200, random_state=42)

# 10折交叉验证
skf = StratifiedKFold(n_splits=5)
for train, test in skf.split(X_new, y_new):
    X_train, X_valid = X_new[train], X_new[test]
    y_train, y_valid = y_new[train], y_new[test]

    clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_valid)
    y_valid_pred = categorical_probas_to_classes(y_pred_proba)

    # 保存预测值
    y_true_all.extend(y_valid)
    y_pred_all.extend(y_valid_pred)
    y_pred_proba_all.extend(y_pred_proba[:, 1])  # 正类概率

    # 评估指标
    TP, FP, FN, TN = confusion_matrix(y_valid, y_valid_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    Sp_collection.append(TN / (TN + FP))
    AAC_collection.append((TP + TN) / (TP + TN + FP + FN))
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    MCC_collection.append(MCC)
    auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
    AUC_collection.append(auc)

# 保存为 CSV 文件
results_df = pd.DataFrame({
    'Predicted_Proba': np.round(y_pred_proba_all, 8),
    'True_Label': y_true_all,
    'Predicted_Label': y_pred_all
})
results_df.to_csv(r'E:/Experiment/work/result/10折/ET_DDE.csv', index=False)

# 打印平均 ± 标准差
print("ACC:", round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print("Sn :", round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print("Sp :", round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print("MCC:", round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print("AUC:", round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))


In [ ]:

# 独立测试
import math
import numpy as np
import statistics
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split

# result collection list
BACC_collection = []
AAC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []
AP = []
mean_recall = np.linspace(0, 1, 100)
all_precision = []
base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
interp_tpr_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(10):
    # dataset splitting
    X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=i)
    
    # Create and train Decision Tree classifier
    clf = ExtraTreesClassifier(max_depth= 500, n_estimators = 500)  # Using default parameters
    clf.fit(X_train_whole, y_train_whole)
    
    # Predict on independent test set
    y_pred_score = clf.predict_proba(X_ind_test)
    y_pred = categorical_probas_to_classes(y_pred_score)
    y_true = y_ind_test
    TP, FP, FN, TN = confusion_matrix(y_true, y_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    AAC_collection.append((TP + TN) / (TP + TN + FP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP * TN - FP * FN) / math.pow(((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)), 0.5)
    MCC_collection.append(MCC)
    BACC_collection.append(0.5 * TP / (TP + FN) + 0.5 * TN / (TN + FP))
    auc = roc_auc_score(y_true, y_pred_score[:, 1])
    AUC_collection.append(auc)
    
    
# Print results
print(round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print(round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print(round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print(round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print(round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))

